In [1]:
import numpy as np
import pandas as pd
from csv import reader
import time
import cpca
import jieba
from gensim import corpora, models, similarities

In [11]:
df=pd.read_excel(r'./深交所上市公司公开数据/创业板.xlsx')
df.head()

D:\1.anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,公司代码,公司简称,公司全称,英文名称,注册地址,A股代码,A股简称,A股上市日期,A股总股本,A股流通股本,B股代码,B股 简 称,B股上市日期,B股总股本,B股流通股本,地 区,省 份,城 市,所属行业,公司网址
0,300001,特锐德,青岛特锐德电气股份有限公司,"Qingdao TGOOD Electric Co.,Ltd.",山东省青岛市崂山区松岭路336号,300001,特锐德,2009-10-30,"997,570,075","944,723,457",NaN,NaN,NaN,0,0,华东,山东,青岛市,C 制造业,www.qdtgood.com
1,300002,神州泰岳,北京神州泰岳软件股份有限公司,"Beijing Ultrapower Software Co., LTD",北京市海淀区万泉庄路28号万柳新贵大厦A座6层601室,300002,神州泰岳,2009-10-30,"1,961,091,984","1,629,793,086",NaN,NaN,NaN,0,0,华北,北京,北京市,I 信息技术,www.ultrapower.com.cn
2,300003,乐普医疗,乐普（北京）医疗器械股份有限公司,"Lepu Medical Technology (Beijing) Co.,Ltd.",北京市昌平区超前路37号,300003,乐普医疗,2009-10-30,"1,781,652,921","1,515,120,211",NaN,NaN,NaN,0,0,华北,北京,北京市,C 制造业,www.lepumedical.com
3,300004,南风股份,南方风机股份有限公司,"NanFang Ventilator Co., Ltd.",广东省佛山市南海区小塘三环西路(狮南段)31号,300004,南风股份,2009-10-30,"479,993,598","418,613,597",NaN,NaN,NaN,0,0,华南,广东,佛山市,C 制造业,www.ntfan.com
4,300005,探路者,探路者控股集团股份有限公司,"Toread Holdings Group Co., Ltd.",北京市昌平区科技园区永安路26号609,300005,探路者,2009-10-30,"883,702,186","706,322,276",NaN,NaN,NaN,0,0,华北,北京,北京市,C 制造业,www.toread.com.cn


In [12]:
def read_data():
    df=pd.read_excel('./深交所上市公司公开数据/创业板.xlsx', dtype={'公司代码': 'str'})
    addr_df=df[["公司代码", "公司简称", "注册地址"]]
    addr_df["注册地址"]=addr_df["注册地址"].apply(lambda x:str(x).strip())
    return addr_df

In [13]:
df=read_data()

D:\1.anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\2170895414.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df["注册地址"]=addr_df["注册地址"].apply(lambda x:str(x).strip())


In [22]:
df.head()

,公司代码,公司简称,注册地址
0,300001,特锐德,山东省青岛市崂山区松岭路336号
1,300002,神州泰岳,北京市海淀区万泉庄路28号万柳新贵大厦A座6层601室
2,300003,乐普医疗,北京市昌平区超前路37号
3,300004,南风股份,广东省佛山市南海区小塘三环西路(狮南段)31号
4,300005,探路者,北京市昌平区科技园区永安路26号609


In [23]:
location_str=[]
for i in df['注册地址']:
        location_str.append(i.strip())

In [24]:
addr_cp = cpca.transform(location_str)

In [25]:
addr_cp

,省,市,区,地址,adcode
0,山东省,青岛市,崂山区,松岭路336号,370212
1,北京市,市辖区,海淀区,万泉庄路28号万柳新贵大厦A座6层601室,110108
2,北京市,市辖区,昌平区,超前路37号,110114
3,广东省,佛山市,南海区,小塘三环西路(狮南段)31号,440605
4,北京市,市辖区,昌平区,科技园区永安路26号609,110114
...,...,...,...,...,...
809,广东省,深圳市,None,坪山区坑梓街道金沙社区锦绣东路23号新产业生物大厦二十一层,440300
810,广东省,广州市,番禺区,石碁镇海涌路109号（厂房）,440113
811,安徽省,合肥市,庐江县,郭河镇工业区,340124
812,江苏省,苏州市,昆山市,玉山镇紫竹路1689号6号房,320583


In [28]:
addr_cpca=pd.concat([df[["公司代码", "公司简称"]],addr_cp],axis=1)

In [29]:
addr_cpca

,公司代码,公司简称,省,市,区,地址,adcode
0,300001,特锐德,山东省,青岛市,崂山区,松岭路336号,370212
1,300002,神州泰岳,北京市,市辖区,海淀区,万泉庄路28号万柳新贵大厦A座6层601室,110108
2,300003,乐普医疗,北京市,市辖区,昌平区,超前路37号,110114
3,300004,南风股份,广东省,佛山市,南海区,小塘三环西路(狮南段)31号,440605
4,300005,探路者,北京市,市辖区,昌平区,科技园区永安路26号609,110114
...,...,...,...,...,...,...,...
809,300832,新产业,广东省,深圳市,None,坪山区坑梓街道金沙社区锦绣东路23号新产业生物大厦二十一层,440300
810,300833,浩洋股份,广东省,广州市,番禺区,石碁镇海涌路109号（厂房）,440113
811,300835,龙磁科技,安徽省,合肥市,庐江县,郭河镇工业区,340124
812,300836,佰奥智能,江苏省,苏州市,昆山市,玉山镇紫竹路1689号6号房,320583


In [33]:
addr_cpca_1 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']!= '')].dropna()

In [34]:
addr_cpca_1

,公司代码,公司简称,省,市,区,地址,adcode
0,300001,特锐德,山东省,青岛市,崂山区,松岭路336号,370212
1,300002,神州泰岳,北京市,市辖区,海淀区,万泉庄路28号万柳新贵大厦A座6层601室,110108
2,300003,乐普医疗,北京市,市辖区,昌平区,超前路37号,110114
3,300004,南风股份,广东省,佛山市,南海区,小塘三环西路(狮南段)31号,440605
4,300005,探路者,北京市,市辖区,昌平区,科技园区永安路26号609,110114
...,...,...,...,...,...,...,...
806,300829,金丹科技,河南省,周口市,郸城县,金丹大道08号,411625
810,300833,浩洋股份,广东省,广州市,番禺区,石碁镇海涌路109号（厂房）,440113
811,300835,龙磁科技,安徽省,合肥市,庐江县,郭河镇工业区,340124
812,300836,佰奥智能,江苏省,苏州市,昆山市,玉山镇紫竹路1689号6号房,320583


In [36]:
addr_cpca_12=addr_cpca_1[(addr_cpca['地址']!='')].dropna(subset=['地址'])

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\3060782526.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_12=addr_cpca_1[(addr_cpca['地址']!='')].dropna(subset=['地址'])


In [37]:
addr_cpca_12.shape

(511, 7)

In [39]:
addr_cpca_12['省市区'] = addr_cpca_12['省'] + addr_cpca_12['市'] + addr_cpca_12['区']
addr_cpca_12['省市区长度']=addr_cpca_12['省市区'].apply(lambda x: len(x))
count_1 = addr_cpca_12['省市区'].value_counts().reset_index()
count_1= count_1.rename(columns={'index':'省市区', '省市区':'个数'})

In [49]:
count_1 = addr_cpca_12['省市区'].value_counts().reset_index()

In [50]:
count_1

,index,省市区
0,北京市市辖区海淀区,57
1,广东省深圳市南山区,37
2,广东省深圳市宝安区,12
3,浙江省杭州市滨江区,12
4,北京市市辖区朝阳区,12
...,...,...
209,山东省东营市东营区,1
210,江苏省南京市六合区,1
211,浙江省杭州市下城区,1
212,新疆维吾尔自治区昌吉回族自治州昌吉市,1


In [73]:
def get_dataset(addr_df):
    '''
    #对地址做一个标准化处理,其中导入cpca的包进行处理
    '''
    start = time.time()
    
    location_str = []
    for i in addr_df['注册地址']:
        location_str.append(i.strip())
        
    addr_cp = cpca.transform(location_str)
    
    #给结果表拼接唯一识别代码
    e_data = addr_df[["公司代码", "公司简称"]]
    addr_cpca = pd.concat([e_data, addr_cp], axis=1)
    
    #1.区不为空
    addr_cpca_1 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']!= '')]
    addr_cpca_1= addr_cpca_1.dropna()
    
    addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
    addr_cpca_12= addr_cpca_11. dropna(subset=['地址'])
    
    #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
    addr_cpca_12['省市区'] = addr_cpca_12['省'] + addr_cpca_12['市'] + addr_cpca_12['区']
    
    addr_cpca_12['省市区长度']=addr_cpca_12['省市区'].apply(lambda x: len(x))
    count_1 = addr_cpca_12['省市区'].value_counts().reset_index()
    count_1= count_1.rename(columns={'index':'省市区', '省市区':'个数'})
    
    count_delete_1= count_1[count_1['个数']==1]
    dataset_1 = pd.merge(addr_cpca_12, count_delete_1, on = '省市区', how = 'left')
    dataset_1= dataset_1[dataset_1['个数']!=1]
    
    #2.区为空  
    addr_cpca_2 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']== '')]
    addr_cpca_2 = addr_cpca_2.dropna()
    
    addr_cpca_21= addr_cpca_2[(addr_cpca['地址']!='')]
    addr_cpca_22= addr_cpca_21. dropna(subset=['地址'])
    
    #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
    addr_cpca_22['省市区'] = addr_cpca_22['省'] + addr_cpca_22['市']
    
    addr_cpca_22['省市区长度']=addr_cpca_22['省市区'].apply(lambda x: len(x))
    count_2 = addr_cpca_22['省市区'].value_counts().reset_index()
    count_2= count_2.rename(columns={'index':'省市区', '省市区':'个数'})
    
    count_delete_2 = count_2[count_2['个数']==1]
    dataset_2 = pd.merge(addr_cpca_22, count_delete_2, on = '省市区', how = 'left')
    dataset_2 = dataset_2[dataset_2['个数']!=1]
    
    print("Time used:", (time. time()-start), "s")
    
    return dataset_1, dataset_2

In [74]:
dataset_1, dataset_2=get_dataset(df)

Time used: 0.038425445556640625 s


C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\84002612.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\84002612.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_21= addr_cpca_2[(addr_cpca['地址']!='')]


In [75]:
dataset_1.head()

,公司代码,公司简称,省,市,区,地址,adcode,省市区,省市区长度,个数
1,300002,神州泰岳,北京市,市辖区,海淀区,万泉庄路28号万柳新贵大厦A座6层601室,110108,北京市市辖区海淀区,9,NaN
2,300003,乐普医疗,北京市,市辖区,昌平区,超前路37号,110114,北京市市辖区昌平区,9,NaN
3,300004,南风股份,广东省,佛山市,南海区,小塘三环西路(狮南段)31号,440605,广东省佛山市南海区,9,NaN
4,300005,探路者,北京市,市辖区,昌平区,科技园区永安路26号609,110114,北京市市辖区昌平区,9,NaN
6,300008,天海防务,上海市,市辖区,松江区,莘砖公路518号10幢8层,310117,上海市市辖区松江区,9,NaN


In [69]:
def cal_similar(doc_goal, document, ssim = 0.1):
#def cal_similar(doc_goai, document):
    '''
    分词;计算文本相似度
    doc_goal,短文本,目标文档
    document,多个文本,被比较的多个文档
    '''
    all_doc_list=[]
    for doc in document:
        doc= "".join(doc)
        doc_list=[word for word in jieba.cut(doc)]
        all_doc_list.append(doc_list)

    #目标文档
    doc_goal = "".join(doc_goal)
    doc_goal_list = [word for word in jieba.cut(doc_goal)]
    
    #被比较的多个文档
    dictionary = corpora.Dictionary(all_doc_list)  #先用dictionary方法获词袋
    corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]  #使用doc2bow制作预料库
    
    #目标文档
    doc_goal_vec = dictionary.doc2bow(doc_goal_list)
    tfidf = models.TfidfModel(corpus)#使用TF-DF模型对料库建模
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))#对每个目标文档,分析测文档的相似度
    
    #开始比较
    sims = index[tfidf[doc_goal_vec]]
    #similary= sorted(enumerate(sims),key=lambda item: -item[1])#根据相似度排序
    
    addr_dict={"被比较地址": document, "相似度": list(sims)}
    similary = pd.DataFrame(addr_dict)
    similary["目标地址"] = doc_goal
    similary_data = similary[["目标地址", "被比较地址", "相似度"]]
    similary_data= similary_data[similary_data["相似度"]>=ssim]
    
    return similary_data

def cycle_first(single_data):
    
    single_value = single_data.loc[:,["公司代码","地址"]].values #提取地址
    
    cycle_data = pd. DataFrame([])
    for key, value in enumerate(single_value):
        if key < len(single_data)-1:
            doc_goal=list(value)[1:]
            document=list(single_data["地址"])[key+1:]
            cycle = cal_similar(doc_goal, document, ssim=0)
            cycle['目标地址代码'] = list(single_data["公司代码"])[key]
            cycle['被比较地址代码'] = list(single_data["公司代码"])[key+1:]
            cycle = cycle[["目标地址代码","目标地址", "被比较地址代码", "被比较地址", "相似度"]]
            #print("循环第",key,"个地址,得到表的行数,",len(cycle),",当前子循环计算进度,",key/len(cycle))
        cycle_data = cycle_data.append(cycle)
        cycle_data = cycle_data.drop_duplicates()
   
    return cycle_data


def get_collect(dataset):

    start = time.time()
    
    #获取单个省市区的文档
    collect_data = pd.DataFrame([])
    ssq=list(set(dataset['省市区']))
    for v, word in enumerate(ssq):
        single_data = dataset[dataset['省市区'] == word]
        print("循环第",v,"个省市区地址为:",word,",当前此区地址有:",len(single_data),",当前计算进度为:{:.1f}%" .format(v*100/len(ssq)))
        cycle_data = cycle_first(single_data)
        
        collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
        
        print("Time: %s" %time.ctime())
        print("-----------------------------------------------------------------------")
    
    print("Time used:",(time.time() - start), "s")
    
    return collect_data

In [92]:
single_data=dataset_1[dataset_1['省市区'] == ssq[1]]
single_value = single_data.loc[:,["公司代码","地址"]].values

In [131]:
for key, value in enumerate(single_value):
    doc_goal=list(value)[1:]
    document=list(single_data["地址"])[key+1:]
    all_doc_list=[]
    for doc in document:
        #doc= "".join(doc)
        #print(doc)
        doc_list=[word for word in jieba.cut(doc)]
        all_doc_list.append(doc_list) 
    dictionary = corpora.Dictionary(all_doc_list)  #先用dictionary方法获词袋
    corpus = [dictionary.doc2bow(doc) for doc in all_doc_list] 
    doc_goal = "".join(doc_goal)
    doc_goal_list = [word for word in jieba.cut(doc_goal)]#使用doc2bow制作预料库
    doc_goal_vec = dictionary.doc2bow(doc_goal_list)
    tfidf = models.TfidfModel(corpus)#使用TF-DF模型对料库建模
    #print(tfidf)
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))
    #print(index)
    ssim = 0.5
    sims = index[tfidf[doc_goal_vec]]
    print(sims )
    #similary= sorted(enumerate(sims),key=lambda item: -item[1])#根据相似度排序
    
    addr_dict={"被比较地址": document, "相似度": list(sims)}
    similary = pd.DataFrame(addr_dict)
    similary["目标地址"] = doc_goal
    similary_data = similary[["目标地址", "被比较地址", "相似度"]]
    print(similary_data)
    similary_data= similary_data[similary_data["相似度"]>=ssim]
    print(similary_data)

[0.         0.57735026]
      目标地址           被比较地址      相似度
0  川大路555号  东育路255弄4号3楼B29  0.00000
1  川大路555号  川沙镇川大路508、518号  0.57735
      目标地址           被比较地址      相似度
1  川大路555号  川沙镇川大路508、518号  0.57735
[0.]
             目标地址           被比较地址  相似度
0  东育路255弄4号3楼B29  川沙镇川大路508、518号  0.0
Empty DataFrame
Columns: [目标地址, 被比较地址, 相似度]
Index: []
[]
Empty DataFrame
Columns: [目标地址, 被比较地址, 相似度]
Index: []
Empty DataFrame
Columns: [目标地址, 被比较地址, 相似度]
Index: []


In [120]:
all_doc_list=[['东育路','255', '255','东育路', '弄', '4', '号', '3', '楼', 'B29'], ['川沙镇', '川', '大路', '508', '、', '518', '号']]

In [121]:
dictionary = corpora.Dictionary(all_doc_list) 
dictionary

In [122]:
corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]  #使用doc2bow制作预料库
print(corpus)

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1)], [(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [103]:
all_doc_list

[]

In [85]:
cycle_first(dataset_1[dataset_1['省市区'] == ssq[1]])

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)


,目标地址代码,目标地址,被比较地址代码,被比较地址,相似度
0,300061,川大路555号,300253,东育路255弄4号3楼B29,0.00000
1,300061,川大路555号,300272,川沙镇川大路508、518号,0.57735
0,300253,东育路255弄4号3楼B29,300272,川沙镇川大路508、518号,0.00000


In [84]:
ssq=list(set(dataset_1['省市区']))
ssq[1]

'上海市市辖区浦东新区'

In [83]:
   for v, word in enumerate(ssq):
        single_data = dataset_1[dataset_1['省市区'] == word]
        print(single_data)

       公司代码  公司简称    省    市    区                          地址  adcode  \
1    300002  神州泰岳  北京市  市辖区  海淀区       万泉庄路28号万柳新贵大厦A座6层601室  110108   
19   300034  钢研高纳  北京市  市辖区  海淀区                    大柳树南村19号  110108   
26   300045  华力创通  北京市  市辖区  海淀区   东北旺西路8号院中关村软件园乙18号楼创通科技大厦  110108   
35   300065   海兰信  北京市  市辖区  海淀区             地锦路7号院10号楼5层501  110108   
38   300070   碧水源  北京市  市辖区  海淀区            生命科学园路23-2号碧水源大厦  110108   
40   300072  三聚环保  北京市  市辖区  海淀区          人大北路33号1号楼大行基业大厦9层  110108   
43   300075  数字政通  北京市  市辖区  海淀区  西直门北大街32号枫蓝国际中心1号楼18层1805室  110108   
45   300079  数码科技  北京市  市辖区  海淀区            上地信息产业基地开拓路15号1幢  110108   
76   300139  晓程科技  北京市  市辖区  海淀区         西三环北路87号国际财经中心D座503  110108   
80   300150  世纪瑞尔  北京市  市辖区  海淀区               创业路8号3号楼6层3-9  110108   
84   300157  恒泰艾普  北京市  市辖区  海淀区              丰秀中路3号院4号楼401室  110108   
98   300182  捷成股份  北京市  市辖区  海淀区             知春路1号学院国际大厦709室  110108   
104  300191  潜能恒信  北京市  市辖区  海淀区              北洼路30号1号楼A416室  11

In [79]:
get_collect(dataset_1)

循环第 0 个省市区地址为: 北京市市辖区海淀区 ,当前此区地址有: 57 ,当前计算进度为:0.0%


C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 1 个省市区地址为: 上海市市辖区浦东新区 ,当前此区地址有: 3 ,当前计算进度为:1.3%
Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 2 个省市区地址为: 浙江省温州市乐清市 ,当前此区地址有: 3 ,当前计算进度为:2.6%
Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 3 个省市区地址为: 江苏省苏州市昆山市 ,当前此区地址有: 8 ,当前计算进度为:3.9%
Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 4 个省市区地址为: 山东省青岛市市北区 ,当前此区地址有: 2 ,当前计算进度为:5.2%
Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 5 个省市区地址为: 广东省惠州市惠阳区 ,当前此区地址有: 2 ,当前计算进度为:6.5%
Time: Mon Feb 20 20:38:54 2023
-----------------------------------------------------------------------
循环第 6 个省市区地址为: 广东省深圳市龙岗区 ,当前此区地址有: 9 ,当前计算进度为:7.8%
Time: Mon Feb 20 20:38:54 2023
--------------------------------------------

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

...

Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 60 个省市区地址为: 江苏省南京市浦口区 ,当前此区地址有: 2 ,当前计算进度为:77.9%
Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 61 个省市区地址为: 广东省深圳市罗湖区 ,当前此区地址有: 2 ,当前计算进度为:79.2%
Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 62 个省市区地址为: 浙江省绍兴市上虞区 ,当前此区地址有: 2 ,当前计算进度为:80.5%
Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 63 个省市区地址为: 江苏省常州市武进区 ,当前此区地址有: 4 ,当前计算进度为:81.8%
Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 64 个省市区地址为: 北京市市辖区门头沟区 ,当前此区地址有: 2 ,当前计算进度为:83.1%
Time: Mon Feb 20 20:38:56 2023
-----------------------------------------------------------------------
循环第 65 个省市区地址为: 广东省深圳市南山区 ,当前此区地址有: 37 ,当前计算进度为:84.4%


C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 66 个省市区地址为: 江苏省无锡市新吴区 ,当前此区地址有: 2 ,当前计算进度为:85.7%
Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 67 个省市区地址为: 四川省成都市青羊区 ,当前此区地址有: 2 ,当前计算进度为:87.0%
Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 68 个省市区地址为: 北京市市辖区丰台区 ,当前此区地址有: 6 ,当前计算进度为:88.3%
Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 69 个省市区地址为: 四川省成都市武侯区 ,当前此区地址有: 4 ,当前计算进度为:89.6%
Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 70 个省市区地址为: 广东省广州市黄埔区 ,当前此区地址有: 4 ,当前计算进度为:90.9%
Time: Mon Feb 20 20:38:57 2023
-----------------------------------------------------------------------
循环第 71 个省市区地址为: 湖南省长沙市岳麓区 ,当前此区地址有: 2 ,当前计算进度为:92.2%
Time: Mon Feb 20 20:38:57 2023
---------------------------------

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\

,目标地址代码,目标地址,被比较地址代码,被比较地址,相似度
0,300002,万泉庄路28号万柳新贵大厦A座6层601室,300034,大柳树南村19号,0.009497
1,300002,万泉庄路28号万柳新贵大厦A座6层601室,300045,东北旺西路8号院中关村软件园乙18号楼创通科技大厦,0.020633
2,300002,万泉庄路28号万柳新贵大厦A座6层601室,300065,地锦路7号院10号楼5层501,0.013048
3,300002,万泉庄路28号万柳新贵大厦A座6层601室,300070,生命科学园路23-2号碧水源大厦,0.030322
4,300002,万泉庄路28号万柳新贵大厦A座6层601室,300072,人大北路33号1号楼大行基业大厦9层,0.042289
...,...,...,...,...,...
1,300385,蠡湖大道2020号,300694,胡埭镇天竹路2号,0.000000
2,300385,蠡湖大道2020号,300782,建筑西路777号A3幢11层,0.000000
0,300555,五三零大厦1号十八层1802室,300694,胡埭镇天竹路2号,0.000000
1,300555,五三零大厦1号十八层1802室,300782,建筑西路777号A3幢11层,0.000000


In [70]:
def run_(par = 0):
    #调用上述函数
    addr_df = read_data()
    dataset_1, dataset_2 = get_dataset(addr_df)
    #dataset. to_csv("../data/addr_data/document_address.csv", index =False)
    #dataset. to_csv( ". /data/addr_data/document_address. csv", index False)
    collect_data_1 = get_collect(dataset_1)
    collect_data_2 = get_collect(dataset_2)
    collect_data = pd.concat([collect_data_1, collect_data_2], axis=0)
    collect_data = collect_data[collect_data["相似度"]>=par].sort_values(by=["相似度"], ascending=[False])
    
    collect_data["相似度"] = collect_data["相似度"].apply(lambda x: ('%.2f' % x))

    return collect_data

In [71]:
collect_data = run_(par = 0.1)

D:\1.anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\2170895414.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df["注册地址"]=addr_df["注册地址"].apply(lambda x:str(x).strip())
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\84002612.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\84002612.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_21= addr_cpca_2[(add

Time used: 0.05954623222351074 s
循环第 0 个省市区地址为: 北京市市辖区海淀区 ,当前此区地址有: 57 ,当前计算进度为:0.0%


Dumping model to file cache C:\Users\jimen\AppData\Local\Temp\jieba.cache
Loading model cost 1.269 seconds.
Prefix dict has been built successfully.
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
C:\Users\jimen\

...

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\

Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 59 个省市区地址为: 上海市市辖区闵行区 ,当前此区地址有: 5 ,当前计算进度为:76.6%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 60 个省市区地址为: 江苏省南京市浦口区 ,当前此区地址有: 2 ,当前计算进度为:77.9%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 61 个省市区地址为: 广东省深圳市罗湖区 ,当前此区地址有: 2 ,当前计算进度为:79.2%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 62 个省市区地址为: 浙江省绍兴市上虞区 ,当前此区地址有: 2 ,当前计算进度为:80.5%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 63 个省市区地址为: 江苏省常州市武进区 ,当前此区地址有: 4 ,当前计算进度为:81.8%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 64 个省市区地址为: 北京市市辖区门头沟区 ,当前此区地址有: 2 ,当前计算进度为:83.1%
Time: Mon Feb 20 20:27:58 2023
--------------------------------

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipyker

Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 66 个省市区地址为: 江苏省无锡市新吴区 ,当前此区地址有: 2 ,当前计算进度为:85.7%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 67 个省市区地址为: 四川省成都市青羊区 ,当前此区地址有: 2 ,当前计算进度为:87.0%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 68 个省市区地址为: 北京市市辖区丰台区 ,当前此区地址有: 6 ,当前计算进度为:88.3%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 69 个省市区地址为: 四川省成都市武侯区 ,当前此区地址有: 4 ,当前计算进度为:89.6%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 70 个省市区地址为: 广东省广州市黄埔区 ,当前此区地址有: 4 ,当前计算进度为:90.9%
Time: Mon Feb 20 20:27:58 2023
-----------------------------------------------------------------------
循环第 71 个省市区地址为: 湖南省长沙市岳麓区 ,当前此区地址有: 2 ,当前计算进度为:92.2%
Time: Mon Feb 20 20:27:58 2023
---------------------------------

C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\AppData\Local\Temp\ipykernel_26120\611814455.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cycle_data = cycle_data.append(cycle)
C:\Users\jimen\

In [76]:
collect_data

,目标地址代码,目标地址,被比较地址代码,被比较地址,相似度
2,300531,粤海街道学府路63号高新区联合总部大厦36楼,300572,粤海街道学府路63号高新区联合总部大厦35楼,0.91
18,300075,西直门北大街32号枫蓝国际中心1号楼18层1805室,300332,西直门北大街32号枫蓝国际中心2号楼906室,0.90
0,300350,华富街道莲花一村社区皇岗路5001号深业上城（南区）T2栋4308,300377,华富街道莲花一村社区皇岗路5001号深业上城南区T2栋3701,0.86
2,300197,坂田街道雪岗路2018号天安云谷产业园一期3栋B座20层2002单元,300389,坂田街道雪岗路2018号天安云谷产业园一期3栋A座18、19、20层,0.85
18,300252,粤海街道高新区社区高新南九道10号深圳湾科技生态园10栋B座26层,300689,粤海街道高新区社区高新南九道10号深圳湾科技生态园10栋B3401-B3404,0.83
...,...,...,...,...,...
38,300191,北洼路30号1号楼A416室,300674,高梁桥斜街42号院1号楼5层1-519,0.10
2,300546,粤海街道深圳市软件产业基地1栋C座9层,300615,茶光路波顿科技园B栋万科云创17层05号,0.10
3,300008,莘砖公路518号10幢8层,300501,洞泾镇蔡家浜路18号,0.10
42,300045,东北旺西路8号院中关村软件园乙18号楼创通科技大厦,300542,蓝靛厂东路2号院金源时代商务中心2号楼B座8层,0.10
